## Homework 08:  Parallel Programming 01

## Due Date: Apr 12, 2023, 11:59pm

#### Firstname Lastname: Seonhye Yang

#### E-mail: sy3420@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (50p)**

Write an MPI program `sol08pr01.py` that does the following for some arbitrary number of processes $N \geq 2$. Here the number of processes $N$ is given as `N` while calling the code `sol08pr01.py` as: 

`mpirun -n N python3 sol08pr01.py`


Every process will contain one buffer with one integer variable, each of which is initialized to $0$.

For $r=0, 1, \dots, N - 1$, Process $r$ updates its buffer to the value received by $r-1$ (this should only be done for $r \geq 1$), then it squares its rank $r$, adds the result $r^2$ to the value of its own buffer, and then sends the sum to Process $r + 1$. Note that for $r=N-1$ the result will be sent to Process $0$, i.e. by convention, Process $N$ is the same as Process $0$. At the end Process $0$ prints the received value. 

Provide results for: $N = 10$, $N = 15$, $N = 20$, $N = 25$. These are probably more than the available processes on your machine: you can use the option `--oversubscribe` in `mpirun` to let MPI run things anyway.



**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code. 



In [189]:
%%writefile sol08pr01.py
#####
# Finding the piece-wise hermite polynomial interpolation of a set of points
# MPI version
#####
from time import time
import numpy as np
from mpi4py import MPI

from mpi4py import MPI

# comm = MPI.COMM_WORLD

# my_rank = comm.Get_rank()
# p = comm.Get_size()

# if my_rank != 0:
#     message = 'Hello from the other rank {}'.format(my_rank)
#     comm.send(message, dest = 0)

# else:
#     for pid in range(1,p):
#         message = comm.recv(source = pid)
#         print('Process {} receives message: {}'.format(my_rank, message))

# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
name = MPI.Get_processor_name()

buffer = np.array([0])

if rank == 0:
    #when r is 0, send the value to r is 1
    data = buffer
    comm.Send(data, dest = 1)
    
    #receives the value from N-1
    comm.Recv(buffer, source = size - 1)
    print('Processor name is', name, "rank:", rank, 'and', "value:", buffer[0])


for i in range(size-1):
    comm.Recv(buffer, source = rank-1)
    buffer = buffer+rank**2
    comm.Send(buffer, dest = (rank+1)% size)
    
    
    

Overwriting sol08pr01.py


In [190]:
!mpiexec -n 10 python sol08pr01.py

Processor name is Seonhyes-MacBook-Pro.local rank: 0 and value: 285


In [191]:
!mpiexec -n 15 python sol08pr01.py  

Processor name is Seonhyes-MacBook-Pro.local rank: 0 and value: 1015


In [192]:
!mpiexec -n 20 python sol08pr01.py

Processor name is Seonhyes-MacBook-Pro.local rank: 0 and value: 2470


In [172]:
!mpiexec -n 25 python sol08pr01.py

Processor name is Seonhyes-MacBook-Pro.local rank: 0 and value: 4900


---

**Problem 2 (50p)**

Write an MPI program that does the following. There are two processes 0 and 1 that have to exchange $T=10$ messages.  


Process 0 initially reads two float variables from the standard input, call them $x, y$, and must ensure $x \neq 0$ and $y \neq 0$. For example this can be done as:

```
import sys


for line in sys.stdin:
    x = float(line)        
    if x != 0.0:
        break
for line in sys.stdin:
    y = float(line)        
    if y != 0.0:
        break
```


Both Process 0 and Process 1 will carry main results in an element that is part of a process buffer and called $p$. The value in $p$ is initially set to $1$. 


Now the exchange of messages is as follows.


0. Message00: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

1. Message01: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

2. Message01: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

3. Message02: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.


etc.

8. Message08: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

9. Message09: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

Finally, Process 0 prints the value in $p$ as a final result. 


Write the code that implements the protocol above. Additionally, provide results for: $(x, y) = (2, 4)$, $(x, y) = (1, 3)$, $(x, y) = (5, 7)$ and $(x, y) = (5, 10)$.


**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code.



In [183]:
%%writefile problem2.py

import sys
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

'''Message00: Process 0 multiplies its own value in  𝑝
  by  𝑥
  and sends the whole buffer to Process 1.

Message01: Process 1 divides its own value in  𝑝
  by  𝑦
  and sends the whole buffer to Process 0.

Message01: Process 0 multiplies its own value in  𝑝
  by  𝑥
  and sends the whole buffer to Process 1.

Message02: Process 1 divides its own value in  𝑝
  by  𝑦
  and sends the whole buffer to Process 0.

etc.

Message08: Process 0 multiplies its own value in  𝑝
  by  𝑥
  and sends the whole buffer to Process 1.

Message09: Process 1 divides its own value in  𝑝
  by  𝑦
  and sends the whole buffer to Process 0.'''

#buffer = [p = 1, x, y]
buffer = np.array([1, float(sys.argv[1]), float(sys.argv[2])])

#since T = 10
T = 10
for i in range(T):
    if rank == 1:
        #receives from process 0 through tag 0
        comm.Recv(buffer, source = 0, tag = 0)
        #divides its own value in  𝑝 by  𝑦
        buffer[2] = buffer[2]/buffer[1]
        #sends to process 0 through tag 1
        comm.Send(buffer, dest = 0, tag = 1)
        
    if rank == 0:
#         comm.Recv(buffer, source = 1, tag = 1)
        # multiplies its own value in  𝑝 by  𝑥
        buffer[2] = buffer[0]*buffer[2]
        #sends to process 1 through tag 0
        comm.Send(buffer, dest = 1, tag = 0)
        #receives from process 1 through tag 1
        comm.Recv(buffer, source = 1, tag = 1)
        
        if i==9:
            #final result of p
            print(f"p: {buffer[2]}")
        

Overwriting problem2.py


In [184]:
!mpiexec -n 2 python problem2.py 2 4

p: 0.00390625


In [157]:
!mpiexec -n 2 python problem2.py 1 3

p: 3.0


In [154]:
!mpiexec -n 2 python problem2.py 5 7

p: 7.167999999999999e-07


In [141]:
!mpiexec -n 2 python problem2.py 5 10

p: 1.0240000000000003e-06
